In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


In [3]:
train_dir = 'C:/Users/DELL/Music/project_atomcamp_final/food11/train'
test_dir = 'C:/Users/DELL/Music/project_atomcamp_final/food11/test'

In [5]:
batch_size = 32
img_height = 180
img_width = 180

In [7]:
def preprocess(image, label):
    image = tf.image.resize(image, [img_height, img_width])
    image = tf.cast(image, tf.float32) / 255.0  # Normalize images to [0, 1]
    return image, label

In [9]:
def augment(image, label):
    image = tf.image.random_flip_left_right(image)   # Randomly flip image horizontally
    image = tf.image.random_flip_up_down(image)     # Randomly flip image vertically
    image = tf.image.random_brightness(image, max_delta=0.1)  # Randomly change brightness
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)  # Randomly change contrast
    image = tf.image.random_saturation(image, lower=0.9, upper=1.1)  # Randomly change saturation
    image = tf.image.random_hue(image, max_delta=0.1)  # Randomly change hue
    return image, label

In [11]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
).map(preprocess).map(augment)

Found 9900 files belonging to 11 classes.


In [13]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
).map(preprocess)

Found 1100 files belonging to 11 classes.


In [15]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(11, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' if labels are one-hot encoded
              metrics=['accuracy'])

# Print the model summary
model.summary()


C:\Users\DELL\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 18, 18, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 10368)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     5,308,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 11)             │         5,643 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,555,403 (21.19 MB)

 Trainable params: 5,555,403 (21.19 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
train_dataset = train_dataset.shuffle(buffer_size=1000)

history = model.fit(
    train_dataset,
    epochs=15,
    validation_data=test_dataset
)


Epoch 1/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 924s 2s/step - accuracy: 0.1272 - loss: 2.3483 - val_accuracy: 0.1509 - val_loss: 2.2878
Epoch 2/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 654s 2s/step - accuracy: 0.1687 - loss: 2.2740 - val_accuracy: 0.1945 - val_loss: 2.2513
Epoch 3/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 1397s 4s/step - accuracy: 0.2031 - loss: 2.2212 - val_accuracy: 0.1909 - val_loss: 2.2589
Epoch 4/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 874s 2s/step - accuracy: 0.2170 - loss: 2.1988 - val_accuracy: 0.2455 - val_loss: 2.1657
Epoch 5/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 910s 3s/step - accuracy: 0.2635 - loss: 2.1270 - val_accuracy: 0.2882 - val_loss: 2.1158
Epoch 6/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 799s 2s/step - accuracy: 0.2870 - loss: 2.0588 - val_accuracy: 0.3018 - val_loss: 2.0441
Epoch 7/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 827s 2s/step - accuracy: 0.3216 - loss: 1.9646 - val_accuracy: 0.2664 - val_loss: 2.1169
Epoch 8/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 473s 1s/step - accuracy: 0.3401 - loss: 1.9120 - val_acc

In [19]:
model.save('food_classification_model.h5')